In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD, Adam
import math
from sklearn.metrics import mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset = pd.read_csv('/content/drive/MyDrive/Marek/Uni/Seminar_25/time-series_data.csv')
dataset.info()
dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150150 entries, 0 to 150149
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   record_ID        150150 non-null  int64  
 1   week             150150 non-null  object 
 2   store_id         150150 non-null  int64  
 3   sku_id           150150 non-null  int64  
 4   total_price      150149 non-null  float64
 5   base_price       150150 non-null  float64
 6   is_featured_sku  150150 non-null  int64  
 7   is_display_sku   150150 non-null  int64  
 8   units_sold       150150 non-null  int64  
dtypes: float64(2), int64(6), object(1)
memory usage: 10.3+ MB


,record_ID,week,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold
0,1,17/01/11,8091,216418,99.0375,111.8625,0,0,20
1,2,17/01/11,8091,216419,99.0375,99.0375,0,0,28
2,3,17/01/11,8091,216425,133.9500,133.9500,0,0,19
3,4,17/01/11,8091,216233,133.9500,133.9500,0,0,44
4,5,17/01/11,8091,217390,141.0750,141.0750,0,0,52


In [4]:
# Fehlerhafte Reihen entfernen
dataset.dropna(inplace=True)

In [ ]:
#Spalte 'week' in datetime konvertieren
dataset['week'] = pd.to_datetime(dataset['week'], format='%d/%m/%y')

In [ ]:
dataset.set_index('week', inplace=True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 150149 entries, 2011-01-17 to 2013-07-09
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   record_ID        150149 non-null  int64  
 1   store_id         150149 non-null  int64  
 2   sku_id           150149 non-null  int64  
 3   total_price      150149 non-null  float64
 4   base_price       150149 non-null  float64
 5   is_featured_sku  150149 non-null  int64  
 6   is_display_sku   150149 non-null  int64  
 7   units_sold       150149 non-null  int64  
dtypes: float64(2), int64(6)
memory usage: 10.3 MB


In [ ]:
# Vorbereitung der Variablen

numeric_features = ['units_sold', 'total_price', 'base_price']
categorical_features = ['store_id', 'sku_id']

# Label Encoding für kategoriale Features
le_store = LabelEncoder()
le_sku = LabelEncoder()

dataset['store_id_encoded'] = le_store.fit_transform(dataset['store_id'])
dataset['sku_id_encoded'] = le_sku.fit_transform(dataset['sku_id'])

# Neue Features für die Sequenzierung, inklusive der binären Variablen
features = numeric_features + ['is_featured_sku', 'is_display_sku', 'store_id_encoded', 'sku_id_encoded']

In [ ]:
# Aufteilung des Datensatzes

# Schritt 1: Eindeutige 'week'-Werte extrahieren
unique_weeks = dataset.index.unique()
n_weeks = len(unique_weeks)

weeks_train = int(0.5 * n_weeks)
weeks_test = int(0.75 * n_weeks)

# Schritt 2: Definiere die Grenzen für die Sets
train_weeks = unique_weeks[:weeks_train]
val_weeks = unique_weeks[weeks_train:weeks_test]
test_weeks = unique_weeks[weeks_test:]

# Schritt 3: Daten in die jeweiligen Sets aufteilen
train_set = dataset[dataset.index.isin(train_weeks)]
print("Trainingsset:")
print(f"Anzahl der Reihen: {train_set.shape[0]}, mögliche Länge der Sequenzen: {len(train_weeks)}")

val_set = dataset[dataset.index.isin(val_weeks)]
print("Validierungs-Set")
print(f"Anzahl der Reihen: {val_set.shape[0]}, mögliche Länge der Sequenzen: {len(val_weeks)}")

test_set = dataset[dataset.index.isin(test_weeks)]
print("Test-Set:")
print(f"Anzahl der Reihen: {test_set.shape[0]}, mögliche Länge der Sequenzen: {len(test_weeks)}")

Trainingsset:
Anzahl der Reihen: 75075, mögliche Länge der Sequenzen: 65
Validierungs-Set
Anzahl der Reihen: 36960, mögliche Länge der Sequenzen: 32
Test-Set:
Anzahl der Reihen: 38114, mögliche Länge der Sequenzen: 33


In [ ]:
# Skalierer nur für die numerischen Features (ohne binäre)
scalers = {feature: MinMaxScaler() for feature in numeric_features}

for feature in numeric_features:
  scalers[feature].fit(train_set[feature].values.reshape(-1, 1))

In [ ]:
print("Trainings-Set:")
train_set.info()
train_set.head()

Trainings-Set:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 75075 entries, 2011-01-17 to 2012-04-10
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   record_ID         75075 non-null  int64  
 1   store_id          75075 non-null  int64  
 2   sku_id            75075 non-null  int64  
 3   total_price       75075 non-null  float64
 4   base_price        75075 non-null  float64
 5   is_featured_sku   75075 non-null  int64  
 6   is_display_sku    75075 non-null  int64  
 7   units_sold        75075 non-null  int64  
 8   store_id_encoded  75075 non-null  int64  
 9   sku_id_encoded    75075 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 6.3 MB


,record_ID,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,store_id_encoded,sku_id_encoded
week,,,,,,,,,,
2011-01-17,1,8091,216418,99.0375,111.8625,0,0,20,3,1
2011-01-17,2,8091,216419,99.0375,99.0375,0,0,28,3,2
2011-01-17,3,8091,216425,133.9500,133.9500,0,0,19,3,3
2011-01-17,4,8091,216233,133.9500,133.9500,0,0,44,3,0
2011-01-17,5,8091,217390,141.0750,141.0750,0,0,52,3,5


In [ ]:
print("Validierungs-Set:")
val_set.info()
val_set.head()

Validierungs-Set:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 36960 entries, 2012-04-17 to 2012-11-20
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   record_ID         36960 non-null  int64  
 1   store_id          36960 non-null  int64  
 2   sku_id            36960 non-null  int64  
 3   total_price       36960 non-null  float64
 4   base_price        36960 non-null  float64
 5   is_featured_sku   36960 non-null  int64  
 6   is_display_sku    36960 non-null  int64  
 7   units_sold        36960 non-null  int64  
 8   store_id_encoded  36960 non-null  int64  
 9   sku_id_encoded    36960 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 3.1 MB


,record_ID,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,store_id_encoded,sku_id_encoded
week,,,,,,,,,,
2012-04-17,106227,8091,216418,104.0250,104.0250,0,0,40,3,1
2012-04-17,106228,8091,216419,106.1625,106.1625,0,0,43,3,2
2012-04-17,106229,8091,216425,117.5625,132.5250,0,0,47,3,3
2012-04-17,106230,8091,216233,118.2750,133.9500,0,0,46,3,0
2012-04-17,106231,8091,217390,139.6500,168.1500,0,0,36,3,5


In [ ]:
grouped_val = val_set.groupby(['store_id', 'sku_id'])
print(f"Anzahl der Gruppen im val_set: {len(grouped_val)}")

Anzahl der Gruppen im val_set: 1155


In [ ]:
print("Test-Set:")
test_set.info()
test_set.head()

Test-Set:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 38114 entries, 2012-11-27 to 2013-07-09
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   record_ID         38114 non-null  int64  
 1   store_id          38114 non-null  int64  
 2   sku_id            38114 non-null  int64  
 3   total_price       38114 non-null  float64
 4   base_price        38114 non-null  float64
 5   is_featured_sku   38114 non-null  int64  
 6   is_display_sku    38114 non-null  int64  
 7   units_sold        38114 non-null  int64  
 8   store_id_encoded  38114 non-null  int64  
 9   sku_id_encoded    38114 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 3.2 MB


,record_ID,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,store_id_encoded,sku_id_encoded
week,,,,,,,,,,
2012-11-27,158648,8091,216418,88.3500,98.3250,0,0,15,3,1
2012-11-27,158649,8091,216419,96.1875,96.1875,0,0,31,3,2
2012-11-27,158650,8091,216425,119.7000,119.7000,0,0,22,3,3
2012-11-27,158651,8091,216233,118.2750,118.2750,0,0,28,3,0
2012-11-27,158652,8091,217390,141.0750,141.0750,0,0,14,3,5


In [ ]:
# Automatisierte Erstellung der Input Sequenzen

In [ ]:
# Funktion zur Erstellung von Sequenzen
def create_sequences(dataset, features, numeric_features, scalers, timesteps):
    X_all = []
    y_all = []

    grouped = dataset.groupby(['store_id', 'sku_id'])

    for (store_id, sku_id), group in grouped:
        group = group.sort_index()

        # Daten extrahieren
        data = group[features].values

        # Skalieren der numerischen Features
        for feature in numeric_features:
            index = features.index(feature)
            data[:, index] = scalers[feature].transform(data[:, index].reshape(-1, 1)).flatten()

        # Kategoriale Features bleiben als Integer

        for i in range(timesteps, len(data)):
            X_seq = data[i - timesteps:i]
            X_all.append(X_seq)
            y_all.append(data[i, 0])  # units_sold ist der erste Wert in features

    X_array = np.array(X_all)
    y_array = np.array(y_all)
    return X_array, y_array

In [ ]:
# Liste der Fenstergrößen
window_sizes = [10, 15, 20, 25, 30]

# Dictionary, um die erzeugten Datensätze zu speichern
train_sets = {}
val_sets = {}

for window in window_sizes:
    X_train, y_train = create_sequences(train_set, features, numeric_features, scalers, window)
    train_sets[window] = {'X': X_train, 'y': y_train}

    X_val, y_val = create_sequences(val_set, features, numeric_features, scalers, window)
    val_sets[window] = {'X': X_val, 'y': y_val}

    print(f"Fenstergröße {window}:")
    print(f"Trainings-Sequenzen: {X_train.shape}, Validierungs-Sequenzen: {X_val.shape}.")

Fenstergröße 10:
Trainings-Sequenzen: (63525, 10, 7), Validierungs-Sequenzen: (25410, 10, 7).
Fenstergröße 15:
Trainings-Sequenzen: (57750, 15, 7), Validierungs-Sequenzen: (19635, 15, 7).
Fenstergröße 20:
Trainings-Sequenzen: (51975, 20, 7), Validierungs-Sequenzen: (13860, 20, 7).
Fenstergröße 25:
Trainings-Sequenzen: (46200, 25, 7), Validierungs-Sequenzen: (8085, 25, 7).
Fenstergröße 30:
Trainings-Sequenzen: (40425, 30, 7), Validierungs-Sequenzen: (2310, 30, 7).


In [ ]:
erste_sequenz_train = train_sets[10]['X'][0]
print(erste_sequenz_train)

[[0.03930435 0.18958032 0.15535445 0.         0.         0.
  0.        ]
 [0.02991304 0.18813314 0.15384615 0.         0.         0.
  0.        ]
 [0.0466087  0.18813314 0.15384615 0.         0.         0.
  0.        ]
 [0.03965217 0.18813314 0.15384615 0.         0.         0.
  0.        ]
 [0.032      0.18668596 0.15233786 0.         0.         0.
  0.        ]
 [0.05182609 0.15050651 0.15233786 0.         0.         0.
  0.        ]
 [0.04626087 0.15050651 0.15233786 0.         0.         0.
  0.        ]
 [0.04486957 0.15484805 0.15233786 0.         0.         0.
  0.        ]
 [0.05773913 0.15195369 0.15233786 0.         0.         0.
  0.        ]
 [0.04347826 0.15629522 0.15233786 0.         0.         0.
  0.        ]]


In [ ]:
# Optimierung_1

# Optionen für Fenstergrößen
windows = [10, 15, 20, 25, 30]

all_train_losses = {}
all_val_losses = {}

for window in windows:
    # Zugriff auf die Trainings- und Validierungsdaten für die aktuelle Fenstergröße
    X_tr = train_sets[window]['X']
    y_tr = train_sets[window]['y']
    X_v = val_sets[window]['X']
    y_v = val_sets[window]['y']

    print(f"Modell für Sequenzen der Länge {window}")

    # Modell initialisieren
    model = Sequential()

    # Input Layer + LSTM Hidden Layer
    model.add(LSTM(units=10, input_shape=(X_tr.shape[1], X_tr.shape[2])))

    # Output Layer mit linearer Aktivierungsfunktion
    model.add(Dense(1, activation='linear'))

    # Modell kompilieren
    model.compile(optimizer=SGD(learning_rate=0.01), loss='mean_squared_error')

    # Modell trainieren
    history = model.fit(X_tr, y_tr, epochs=10, batch_size=64, validation_data=(X_v, y_v))

    # Verlaufsdaten speichern
    train_losses = history.history['loss']
    val_losses = history.history['val_loss']

    # Speichern in Dictionaries, Schlüssel z.B. als Tuple
    key = (window)
    all_train_losses[key] = train_losses
    all_val_losses[key] = val_losses

    # Ausgabe pro Modell
    avg_val_loss = np.mean(val_losses)
    best_val_loss = np.min(val_losses)
    print(f"Durchschnittlicher Validierungsverlust: {avg_val_loss:.6f}")
    print(f"Bester Validierungsverlust: {best_val_loss:.6f}")

Modell für Sequenzen der Länge 10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
991/993 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0076

KeyboardInterrupt: 

In [ ]:
best_window = 30
X_tr = train_sets[best_window]['X']
y_tr = train_sets[best_window]['y']
X_v = val_sets[best_window]['X']
y_v = val_sets[best_window]['y']

# Hyperparameter-Optionen
layer_options = [1, 2, 3]
units_options = [32, 64, 128]

all_train_losses = {}
all_val_losses = {}

for n_layers in layer_options:
    for units in units_options:
        print(f"\nTraining: {n_layers} Layer(n), {units} Units pro Layer")

        model = Sequential()

        # Erste Schicht
        model.add(LSTM(units=units, input_shape=(X_tr.shape[1], X_tr.shape[2]), return_sequences=(n_layers > 1)))

        # Zusätzliche Schichten bei Bedarf
        for layer_idx in range(1, n_layers):
            # letzte Schicht mit return_sequences=False
            return_seq = (layer_idx < n_layers - 1)
            model.add(LSTM(units=units, return_sequences=return_seq))

        # Endfolger
        model.add(Dense(1, activation='linear'))

        # Kompilieren mit SGD, Lernrate 0,01
        model.compile(optimizer=SGD(learning_rate=0.01), loss='mean_squared_error')

        # Modell trainieren
        history = model.fit(X_tr, y_tr, epochs=15, batch_size=64, validation_data=(X_v, y_v))

        # Verlaufsdaten speichern
        train_losses = history.history['loss']
        val_losses = history.history['val_loss']

        # Speichern in Dictionaries, Schlüssel z.B. als Tuple
        key = (n_layers, units)
        all_train_losses[key] = train_losses
        all_val_losses[key] = val_losses

        # Ausgabe pro Modell
        avg_val_loss = np.mean(val_losses)
        best_val_loss = np.min(val_losses)
        print(f"Durchschnittlicher Validierungsverlust: {avg_val_loss:.6f}")
        print(f"Bester Validierungsverlust: {best_val_loss:.6f}")


Training: 1 Layer(n), 32 Units pro Layer
Epoch 1/15
632/632 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 0.0049 - val_loss: 7.2330e-04
Epoch 2/15
632/632 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - loss: 8.9062e-04 - val_loss: 5.3614e-04
Epoch 3/15
632/632 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - loss: 7.2521e-04 - val_loss: 4.7904e-04
Epoch 4/15
632/632 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 6.6750e-04 - val_loss: 4.3884e-04
Epoch 5/15
632/632 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - loss: 6.3435e-04 - val_loss: 3.8554e-04
Epoch 6/15
632/632 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - loss: 5.9424e-04 - val_loss: 3.5076e-04
Epoch 7/15
632/632 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 5.5172e-04 - val_loss: 3.2770e-04
Epoch 8/15
632/632 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - loss: 5.5556e-04 - val_loss: 3.1917e-04
Epoch 9/15
632/632 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - loss: 5.5911e-04 - val_loss: 3.0535e-04
Epoch 10/15
632/632 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - loss: 4.9494e-04 - val_loss: 3.0515e-04
Epoch 

In [ ]:
# Weitere Optimierung

In [ ]:
# Grid-Search für Dropout und L2
dropout_rates = [0.2, 0.3, 0.4, 0.5]
l2_values = [0.0001, 0.001, 0.01]

# Dictionaries zum Speichern der Verlaufsdaten und Hyperparameter
all_train_losses = {}
all_val_losses = {}
all_hyperparams = []

for dropout in dropout_rates:
    for l2_reg in l2_values:
        print(f"Training mit Dropout={dropout}, L2={l2_reg}")

        model = Sequential()
        # Erste Schicht
        model.add(LSTM(units=128,
                       input_shape=(X_tr.shape[1], X_tr.shape[2]),
                       kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
                       dropout=dropout,
                       recurrent_dropout=dropout,
                       return_sequences=True))
        # Zweite Schicht
        model.add(LSTM(units=128,
                       kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
                       dropout=dropout,
                       recurrent_dropout=dropout,
                       return_sequences=True))
        # Dritte Schicht
        model.add(LSTM(units=128,
                       kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
                       dropout=dropout,
                       recurrent_dropout=dropout))
        model.add(Dense(1, activation='linear'))

        # Kompilieren
        model.compile(optimizer=SGD(learning_rate=0.01),
                      loss='mean_squared_error')

        # Training mit Verlaufsdaten sammeln
        history = model.fit(X_tr, y_tr, epochs=10, batch_size=64,
                            validation_data=(X_v, y_v))

        # Verlaufsdaten speichern
        train_losses = history.history['loss']
        val_losses = history.history['val_loss']

        # Speichern in Dictionaries, Schlüssel z.B. als Tuple
        key = (n_layers, units)
        all_train_losses[key] = train_losses
        all_val_losses[key] = val_losses

        # Ausgabe pro Modell
        avg_val_loss = np.mean(val_losses)
        best_val_loss = np.min(val_losses)
        print(f"Durchschnittlicher Validierungsverlust: {avg_val_loss:.6f}")
        print(f"Bester Validierungsverlust: {best_val_loss:.6f}")

Training mit Dropout=0.2, L2=0.0001


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 172s 260ms/step - loss: 0.0562 - val_loss: 0.0424
Epoch 2/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 199s 255ms/step - loss: 0.0460 - val_loss: 0.0423
Epoch 3/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 200s 252ms/step - loss: 0.0447 - val_loss: 0.0421
Epoch 4/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 201s 251ms/step - loss: 0.0440 - val_loss: 0.0420
Epoch 5/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 204s 255ms/step - loss: 0.0436 - val_loss: 0.0419
Epoch 6/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 202s 255ms/step - loss: 0.0433 - val_loss: 0.0418
Epoch 7/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 165s 261ms/step - loss: 0.0430 - val_loss: 0.0417
Epoch 8/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 197s 254ms/step - loss: 0.0428 - val_loss: 0.0416
Epoch 9/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 203s 255ms/step - loss: 0.0426 - val_loss: 0.0415
Epoch 10/10
632/632 ━━━━━━━━━━━━━━━━━━━━ 209s 265ms/step - loss: 0.0424 - val_loss: 0.0414
Durchschnittlicher Validierungsverlust: 0.041871
Bester Validierungsverlust: 0.041370
Training mit 

In [ ]:
dropout_rates = [0.2, 0.5]

for dropout in dropout_rates:
  print(f"Training mit Dropout={dropout}")

  model = Sequential()
  # Erste Schicht
  model.add(LSTM(units=128,
                  input_shape=(X_tr.shape[1], X_tr.shape[2]),
                  dropout=dropout,
                  recurrent_dropout=dropout,
                  return_sequences=True))
  # Zweite Schicht
  model.add(LSTM(units=128,
                  dropout=dropout,
                  recurrent_dropout=dropout,
                  return_sequences=True))
  # Dritte Schicht
  model.add(LSTM(units=128,
                  dropout=dropout,
                  recurrent_dropout=dropout))
  model.add(Dense(1, activation='linear'))

  # Kompilieren
  model.compile(optimizer=SGD(learning_rate=0.01),
                loss='mean_squared_error')

  # Training mit Verlaufsdaten sammeln
  history = model.fit(X_tr, y_tr, epochs=10, batch_size=64,
                            validation_data=(X_v, y_v))

  # Verlaufsdaten speichern
  train_losses = history.history['loss']
  val_losses = history.history['val_loss']

  # Ausgabe pro Modell
  avg_val_loss = np.mean(val_losses)
  best_val_loss = np.min(val_losses)
  print(f"Durchschnittlicher Validierungsverlust: {avg_val_loss:.6f}")
  print(f"Bester Validierungsverlust: {best_val_loss:.6f}")

Training mit Dropout=0.2
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


993/993 ━━━━━━━━━━━━━━━━━━━━ 140s 134ms/step - loss: 0.0084 - val_loss: 3.5947e-04
Epoch 2/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 123s 124ms/step - loss: 0.0024 - val_loss: 3.1091e-04
Epoch 3/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 138s 120ms/step - loss: 0.0016 - val_loss: 3.0335e-04
Epoch 4/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 139s 117ms/step - loss: 0.0012 - val_loss: 3.0155e-04
Epoch 5/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 146s 121ms/step - loss: 0.0010 - val_loss: 2.9979e-04
Epoch 6/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 144s 123ms/step - loss: 8.9594e-04 - val_loss: 2.9995e-04
Epoch 7/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 133s 114ms/step - loss: 8.0953e-04 - val_loss: 3.0094e-04
Epoch 8/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 145s 117ms/step - loss: 7.4664e-04 - val_loss: 2.9870e-04
Epoch 9/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 141s 116ms/step - loss: 6.9861e-04 - val_loss: 2.9831e-04
Epoch 10/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 151s 126ms/step - loss: 6.6109e-04 - val_loss: 2.9856e-04
Durchschnittlicher Validierungsverlust: 0.000307
B

TypeError: 'float' object is not subscriptable

In [ ]:
l2_values = [0.0001, 0.01]

for l2_reg in l2_values:
  print(f"Training mit L2={l2_reg}")

  model = Sequential()
  # Erste Schicht
  model.add(LSTM(units=128,
                  input_shape=(X_tr.shape[1], X_tr.shape[2]),
                  kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
                  return_sequences=True))
  # Zweite Schicht
  model.add(LSTM(units=128,
                  kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
                  return_sequences=True))
  # Dritte Schicht
  model.add(LSTM(units=128,
                  kernel_regularizer=tf.keras.regularizers.l2(l2_reg)))
  model.add(Dense(1, activation='linear'))

  # Kompilieren
  model.compile(optimizer=SGD(learning_rate=0.01),
                loss='mean_squared_error')

  # Training mit Verlaufsdaten sammeln
  history = model.fit(X_tr, y_tr, epochs=10, batch_size=64,
                            validation_data=(X_v, y_v))

  # Verlaufsdaten speichern
  train_losses = history.history['loss']
  val_losses = history.history['val_loss']

  # Speichern in Dictionaries, Schlüssel z.B. als Tuple
  key = (l2_reg)
  all_train_losses[key] = train_losses
  all_val_losses[key] = val_losses

  # Ausgabe pro Modell
  avg_val_loss = np.mean(val_losses)
  best_val_loss = np.min(val_losses)
  print(f"Durchschnittlicher Validierungsverlust: {avg_val_loss:.6f}")
  print(f"Bester Validierungsverlust: {best_val_loss:.6f}")

Training mit L2=0.0001
Epoch 1/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 100s 97ms/step - loss: 0.0433 - val_loss: 0.0425
Epoch 2/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 90s 91ms/step - loss: 0.0425 - val_loss: 0.0423
Epoch 3/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 155s 104ms/step - loss: 0.0423 - val_loss: 0.0421
Epoch 4/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 136s 97ms/step - loss: 0.0421 - val_loss: 0.0420
Epoch 5/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 142s 98ms/step - loss: 0.0420 - val_loss: 0.0418
Epoch 6/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 132s 88ms/step - loss: 0.0418 - val_loss: 0.0416
Epoch 7/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 142s 88ms/step - loss: 0.0416 - val_loss: 0.0415
Epoch 8/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 152s 98ms/step - loss: 0.0414 - val_loss: 0.0413
Epoch 9/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 97s 97ms/step - loss: 0.0413 - val_loss: 0.0411
Epoch 10/10
993/993 ━━━━━━━━━━━━━━━━━━━━ 145s 100ms/step - loss: 0.0411 - val_loss: 0.0410
Durchschnittlicher Validierungsverlust: 0.041712
Bester Validierungsverlust: 0.040960


TypeError: 'float' object is not subscriptable